In [1]:
!nvidia-smi

Wed May 18 17:17:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   49C    P8    43W / 300W |     10MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:2E:00.0 Off |                  Off |
| 56%   

In [2]:
import numpy as np
import torch
import torch.nn as nn
from model import FNN
from util import *
from train import *
from torch.autograd import Variable,grad
import time

In [3]:
def output_transform(X):
    X = T_range*nn.Softplus()(X) + T_ref
    return X


def input_transform(X):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X


def PDE(x,y,z,t,net):
    X = torch.concat([x,y,z,t],axis=-1)
    T = net(X)
    
    T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]

    T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x,x,create_graph=True,grad_outputs=torch.ones_like(T_x))[0]
    
    T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y,y,create_graph=True,grad_outputs=torch.ones_like(T_y))[0]
    
    T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z,z,create_graph=True,grad_outputs=torch.ones_like(T_z))[0]
    
    f = rho*Cp*T_t - k*(T_xx+T_yy+T_zz)

    return f


def generate_points(p=[],f=[]):

    t = np.linspace(0,x_max[3],31)

    # boundary points
    bound_x_neg,_ = sampling_uniform(1.,x_min,x_max,'-x',t)
    bound_x_pos,_ = sampling_uniform(1.,x_min,x_max,'+x',t)

    bound_y_neg,_ = sampling_uniform(1.,x_min,x_max,'-y',t)
    bound_y_pos,_ = sampling_uniform(1.,x_min,x_max,'+y',t)

    bound_z_neg,_ = sampling_uniform(1.,x_min,x_max,'-z',t)
    bound_z_pos,_ = sampling_uniform(1.,x_min,x_max,'+z',t)

    bound_z_pos_more = [] # more points for surface flux
    for ti in t:
        if ti<t_end:
            zi,_ = sampling_uniform(.25,
                        [max(x0+ti*v-2*r,x_min[0]),max(x_min[1],y0-2*r),x_min[2]],
                        [min(x0+ti*v+2*r,x_max[0]),min(x_max[1],y0+2*r),x_max[2]],
                        '+z',[ti])
            bound_z_pos_more.append(zi)

    bound_z_pos_more = np.vstack(bound_z_pos_more)
    bound_z_pos = np.vstack((bound_z_pos,bound_z_pos_more))

    ### domain points
    e = 0.05
    domain_pts1,_ = sampling_uniform(2.,
                                     [x_min[0]+e,x_min[1]+e,x_min[2]+e],
                                     [x_max[0]-e,x_max[1]-e,x_max[2]-3.],'domain',t)

    domain_pts2,_ = sampling_uniform(1.,
                                     [x_min[0]+e,x_min[1]+e,x_max[2]-3.+e],
                                     [x_max[0]-e,x_max[1]-e,x_max[2]-1.],'domain',t)

    domain_pts3 = []
    for ti in t:
        di,_ = sampling_uniform(.5,
                                [x_min[0]+e,x_min[1]+e,x_max[2]-1.+e,],
                                [x_max[0]-e,x_max[1]-e,x_max[2]-e],'domain',[ti])
        domain_pts3.append(di)
    domain_pts3 = np.vstack(domain_pts3)
    domain_pts = np.vstack((domain_pts1,domain_pts2,domain_pts3))

    init_pts,_ = sampling_uniform(2.,[x_min[0],x_min[1],x_min[2]],
                                   [x_max[0],x_max[1],x_max[2]],'domain',[0])

    p.extend([torch.tensor(bound_x_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_x_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(init_pts,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(domain_pts,requires_grad=True,dtype=torch.float).to(device)])
    f.extend([['BC','-x'],['BC','+x'],['BC','-y'],['BC','+y'],['BC','-z'],['BC','+z'],['IC',T_ref],['domain']])
    
    return p,f


def BC(x,y,z,t,loc,l=None):
    X = torch.concat([x,y,z,t],axis=-1)
    T = net(X)
    if loc == '-x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_x - h*T - Rboltz*emiss*T**4
    if loc == '+x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_x - h*T - Rboltz*emiss*T**4
    if loc == '-y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_y - h*T - Rboltz*emiss*T**4
    if loc == '+y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_y - h*T - Rboltz*emiss*T**4
    if loc == '-z':
        T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return T_t
    if loc == '+z':
        T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        q = 2*P*eta/torch.pi/r**2*torch.exp(-2*(torch.square(x-x0-v*t)+torch.square(y-y0))/r**2)*(t<=t_end)
        return -k*T_z - h*T + q

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

task = 'bareplate'
x_max = np.array([40., 10.,  6.,  3.])
x_min = np.array([ 0.,  0.,  0.,  0.])
X_max = torch.tensor(x_max,dtype=torch.float).to(device)
X_min = torch.tensor(x_min,dtype=torch.float).to(device)

T_ref = 298.
T_range = 3000.

Cp = 0.5
k = 0.01

x0 = 5. # laser start x
y0 = 5.
r = 1.5 # beam radius
v = 10 # speed
t_end = 3. # laser end time

h = 2e-5
eta = 0.4
P = 500
Rboltz = 5.6704e-14
emiss = 0.3
rho = 8e-3

In [5]:
t = np.linspace(0,x_max[3],31)

# boundary points
bound_x_neg,_ = sampling_uniform(1.,x_min,x_max,'-x',t)
bound_x_pos,_ = sampling_uniform(1.,x_min,x_max,'+x',t)

bound_y_neg,_ = sampling_uniform(1.,x_min,x_max,'-y',t)
bound_y_pos,_ = sampling_uniform(1.,x_min,x_max,'+y',t)

bound_z_neg,_ = sampling_uniform(1.,x_min,x_max,'-z',t)
bound_z_pos,_ = sampling_uniform(1.,x_min,x_max,'+z',t)

bound_z_pos_more = [] # more points for surface flux
for ti in t:
    if ti<t_end:
        zi,_ = sampling_uniform(.25,
                    [max(x0+ti*v-2*r,x_min[0]),max(x_min[1],y0-2*r),x_min[2]],
                    [min(x0+ti*v+2*r,x_max[0]),min(x_max[1],y0+2*r),x_max[2]],
                    '+z',[ti])
        bound_z_pos_more.append(zi)

bound_z_pos_more = np.vstack(bound_z_pos_more)
bound_z_pos = np.vstack((bound_z_pos,bound_z_pos_more))



In [ ]:
iterations = 50000

net = FNN([4,64,64,64,1],nn.Tanh(),in_tf=input_transform,out_tf=output_transform)
net.to(device)

point_sets,flags = generate_points([],[])

l_history = train(net,PDE,BC,point_sets,flags,iterations,lr=5e-4,info_num=100)

It: 0, Loss: 3.758e+03, L_bc: 1.432e+04, L_ic: 7.046e+06, L_data: 0.000e+00, L_PDE: 1.620e+00, Time: 0.10
It: 100, Loss: 8.167e+01, L_bc: 6.704e+01, L_ic: 2.597e+06, L_data: 0.000e+00, L_PDE: 1.051e-02, Time: 9.79
It: 200, Loss: 5.045e+01, L_bc: 4.489e+01, L_ic: 1.569e+06, L_data: 0.000e+00, L_PDE: 6.850e-03, Time: 9.80
It: 300, Loss: 3.494e+01, L_bc: 4.308e+01, L_ic: 9.668e+05, L_data: 0.000e+00, L_PDE: 9.698e-03, Time: 9.81
It: 400, Loss: 2.588e+01, L_bc: 4.294e+01, L_ic: 6.057e+05, L_data: 0.000e+00, L_PDE: 1.108e-02, Time: 9.82
It: 500, Loss: 2.048e+01, L_bc: 4.287e+01, L_ic: 3.904e+05, L_data: 0.000e+00, L_PDE: 1.106e-02, Time: 9.84
It: 600, Loss: 1.722e+01, L_bc: 4.279e+01, L_ic: 2.608e+05, L_data: 0.000e+00, L_PDE: 1.032e-02, Time: 9.80
It: 700, Loss: 1.520e+01, L_bc: 4.273e+01, L_ic: 1.808e+05, L_data: 0.000e+00, L_PDE: 9.345e-03, Time: 9.82
It: 800, Loss: 1.392e+01, L_bc: 4.267e+01, L_ic: 1.298e+05, L_data: 0.000e+00, L_PDE: 8.404e-03, Time: 9.92
It: 900, Loss: 1.307e+01, L_bc

It: 7500, Loss: 8.355e+00, L_bc: 3.132e+01, L_ic: 4.582e+03, L_data: 0.000e+00, L_PDE: 1.637e+00, Time: 10.04
It: 7600, Loss: 8.143e+00, L_bc: 3.025e+01, L_ic: 4.362e+03, L_data: 0.000e+00, L_PDE: 1.888e+00, Time: 10.04
It: 7700, Loss: 7.951e+00, L_bc: 2.935e+01, L_ic: 4.039e+03, L_data: 0.000e+00, L_PDE: 2.051e+00, Time: 10.04
It: 7800, Loss: 7.771e+00, L_bc: 2.858e+01, L_ic: 3.671e+03, L_data: 0.000e+00, L_PDE: 2.143e+00, Time: 10.04
It: 7900, Loss: 7.598e+00, L_bc: 2.786e+01, L_ic: 3.326e+03, L_data: 0.000e+00, L_PDE: 2.200e+00, Time: 10.06
It: 8000, Loss: 7.424e+00, L_bc: 2.717e+01, L_ic: 3.024e+03, L_data: 0.000e+00, L_PDE: 2.219e+00, Time: 10.03
It: 8100, Loss: 7.239e+00, L_bc: 2.647e+01, L_ic: 2.810e+03, L_data: 0.000e+00, L_PDE: 2.205e+00, Time: 10.04
It: 8200, Loss: 7.035e+00, L_bc: 2.571e+01, L_ic: 2.691e+03, L_data: 0.000e+00, L_PDE: 2.159e+00, Time: 10.04
It: 8300, Loss: 6.796e+00, L_bc: 2.483e+01, L_ic: 2.651e+03, L_data: 0.000e+00, L_PDE: 2.087e+00, Time: 10.04
It: 8400, 

In [ ]:
torch.save(net.state_dict(),'../model/{}.pt'.format(task))
np.save('../model/{}.npy'.format(task),l_history)